## Executing a Lattice as an Electron (Sublattice)

Wrap a lattice with the `@electron` decorator to run it as a self-contained unit in another lattice.

### Prerequisites

[Start the Covalent services](./covalent_cli.rst).

### Procedure

1. Write a lattice.
2. Make the lattice an electron using the `@electron` decorator:

In [1]:
import covalent as ct

@ct.electron
def identity(x):
    return x

@ct.electron
def root(x):
    return x ** 0.5

@ct.electron
def square(x):
    return x * x

# A lattice wrapped in an electron: a sublattice
@ct.electron
@ct.lattice
def multiply_3_and_square(x):
    root(x=x)
    return square(x=x * 3)

3. Run the sublattice in another lattice as if it were any other electron:

In [2]:
@ct.lattice
def workflow(a):
    val_1 = identity(x=a)
    return multiply_3_and_square(x=val_1)

Since the sublattice is also a lattice, you can pass constraints to it the same as you would [any other lattice](../coding/add_constraints_to_lattice.ipynb).

4. Dispatch the top-level lattice `workflow` and query the result.

In [3]:
dispatch_id = ct.dispatch(workflow)(a=2)
print("Dispatch id:", dispatch_id)

result = ct.get_result(dispatch_id, wait=True)
print("Result of workflow:", result.result)

Dispatch id: d3b83756-78a4-4079-a3f7-1ae36eb75d64
Result of workflow: 36


5. Find the node ID of the sublattice using the `get_all_node_outputs()` function of the Covalent `Result` object:

In [4]:
result.get_all_node_outputs()

{'identity(0)': <covalent.TransportableObject at 0x106aee6a0>,
 ':parameter:2(1)': <covalent.TransportableObject at 0x106aee610>,
 ':sublattice:multiply_3_and_square(2)': <covalent.TransportableObject at 0x106aee430>}

The number in parenthesis denotes the node ID. For the `identity` node, the ID is `0`. The ID of the sublattice `multiply_3_and_square` is `2`.

6. Use this information to query the sublattice's result using the result's `get_node_result` function:

In [5]:
result.get_node_result(2)

{'node_id': 2,
 'node_name': ':sublattice:multiply_3_and_square',
 'start_time': datetime.datetime(2022, 12, 30, 19, 45, 21, 554108, tzinfo=datetime.timezone.utc),
 'end_time': datetime.datetime(2022, 12, 30, 19, 45, 21, 777670, tzinfo=datetime.timezone.utc),
 'status': Status(STATUS='COMPLETED'),
 'output': <covalent.TransportableObject at 0x106aee430>,
 'error': None,
 'sublattice_result': <covalent._results_manager.result.Result at 0x106aee520>,
 'stdout': None,
 'stderr': None}

The `sublattice_result` value shows that a `Result` object is present. This is just like any other result object; the only difference is that it was returned by a sublattice.

7. Query the sublattice result:

In [6]:
sublattice_result = result.get_node_result(2)["sublattice_result"]

sublattice_result.result

36

8. Optionally, access all of the executions performed by the sublattice using `get_all_node_outputs()`:

In [7]:
sublattice_result.get_all_node_outputs()

{'root(0)': <covalent.TransportableObject at 0x106b1b040>,
 ':parameter:2(1)': <covalent.TransportableObject at 0x106b1b0a0>,
 'square(2)': <covalent.TransportableObject at 0x106b1b340>,
 ':parameter:6(3)': <covalent.TransportableObject at 0x106ad2fa0>}

### See Also

[Adding Constraints to Tasks and Workflows](../coding/add_constraints_to_lattice.ipynb).